# Nonlinear Elasticity

In this notebook we compare three different materials. The linear material law of Hooke, the geometric nonlinear material law of St.Venant-Kirchhoff, and the nonlinear Neo-Hooke material law, which avoids that the material gets fully compressed by penalizing the volume (deformation determinant) going to zero.

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *

We consider the same domain as in the linear elasticity example.

In [ ]:
def GenerateMesh(t=0.1):
    bar = MoveTo(0,-t/2).Rectangle(1,t).Face()
    bar.edges.Min(X).name="left"
    bar.edges.Max(X).name="right"
    bar.edges.Min(X).maxh=t/8
    mesh = Mesh(OCCGeometry(bar, dim=2).GenerateMesh(maxh=t/2.5))
    return mesh
mesh = GenerateMesh()
Draw(mesh);

Remember that with the displacement $u$, the deformation gradient $\boldsymbol{F}$, Cauchy-Green strain tensor $\boldsymbol{C}$, and Green strain tensor $\boldsymbol{E}$ reads
\begin{align*}
\boldsymbol{F} = \nabla u + \boldsymbol{I},\qquad \boldsymbol{C} = \boldsymbol{F}^T\boldsymbol{F},\qquad \boldsymbol{E}=\frac{1}{2}(\boldsymbol{C}-\boldsymbol{I}) = \frac{1}{2}(\nabla u^T\nabla u+\nabla u^T+\nabla u).
\end{align*}

With the Cauchy-Green and Green strain tensor at hand, we define the three material laws by their hyperelastic energy densities 
\begin{align*}
\Psi^{H}(E) &= \mu\,\|\varepsilon(u)\|^2+\frac{\lambda}{2}\mathrm{tr}(\varepsilon(u))^2,\\
\Psi^{VK}(E) &= \mu\,\|E\|^2+\frac{\lambda}{2}\mathrm{tr}(E)^2,\\
\Psi^{NH}(C) &= \frac{\mu}{2}(\mathrm{tr}(C-I)-\log(\det(C))) + \frac{\lambda}{2}(\sqrt{\det(C)}-1)^2.
\end{align*}

In [ ]:
E, nu = 210, 0.2
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

def C(u):
    F = Id(2) + Grad(u)
    return F.trans * F

def Hooke(u):
    E = Sym(Grad(u))
    return mu*InnerProduct(E,E) + lam/2*Trace(E)**2

def StVenantKirchhoff(u):
    E = 0.5*(C(u)-Id(2))
    return mu*InnerProduct(E,E) + lam/2*Trace(E)**2

def NeoHooke(u):
    return mu/2*(Trace(C(u)-Id(2))- log(Det(C(u)))) + lam/2*(Det(Id(2)+Grad(u))-1)**2

Instead of working with the weak form, we can directly use the energy minimization formulation
$$\mathcal{W}(u) = \int_{\Omega} \Psi(u) - f u\,dx \to\min!$$
and let NGSolve compute the first and second variation needed for Newton's method to solve the nonlinear minimization problem for us.

Note, that we only use the trial function $u$ (no test function $v$). Further, we need to write the expression inside Variation() to tell NGSolve that it needs to compute the first and second variation. To help Newton's method to converge, we use the concept of load-stepping. To this end, we scale the right-hand side by a factor in $(0,1]$ and increase it after Newton converged. Then the previously computed solution is used as initial guess for the next load-step.

In [ ]:
factor = Parameter(0)
force = CF( (0,-factor) )

fes = VectorH1(mesh, order=3, dirichlet="left")
u  = fes.TrialFunction()

aNH = BilinearForm(fes, symmetric=True)
aNH += Variation((NeoHooke(u)-force*u)*dx)

gfuNH = GridFunction(fes)

We can construct a simple Newton solver, using automatic differentiation for residual and tangential stiffness. Apply() computes the first variation and AssembleLinearization() the second one. With Energy() the energy is evaluated. We see that it gets minimized during Newton's method.

In [ ]:
def SolveNewton():
    res = gfuNH.vec.CreateVector()
    
    for it in range(10):
        print ("it", it, "energy = ", aNH.Energy(gfuNH.vec))
        aNH.Apply(gfuNH.vec, res)
        aNH.AssembleLinearization(gfuNH.vec)
        inv = aNH.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky") 
        gfuNH.vec.data -= inv*res

Solve the problem and draw the xx-component of the Cauchy-Green tensor, which is greater than 1 if the material gets stretched in x-direction during deformation and small than 1 when compressed.

In [ ]:
factor.Set(0.4)
SolveNewton()
scene = Draw (C(gfuNH)[0,0], mesh, deformation=gfuNH)

We can increase the load-step and solve with the last solution as initial guess.

In [ ]:
factor.Set(factor.Get()+0.4)
solvers.Newton(aNH,gfuNH)
scene.Redraw()

## Compare linear Hooke, St. Venant-Kirchhoff and Neo-Hooke

Next, we compare the three material laws. We start with the linear material law of Hooke.

In [ ]:
def SolveHooke(mesh, order=2):
    fes = VectorH1(mesh, order=order, dirichlet="left")
    u,v = fes.TnT()

    a = BilinearForm(fes, symmetric=True)
    a += Variation( Hooke(u)*dx-force*u*dx)

    gfu = GridFunction(fes)
    
    r = gfu.vec.CreateVector()
    a.Apply(gfu.vec, r)
    a.AssembleLinearization(gfu.vec)

    gfu.vec.data -= a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky")*r
    return gfu

Solve it and draw the deformation determinant $J=\det(\boldsymbol{F})= \nabla u+\boldsymbol{I}$. We see that for larger deformations, the linear material law of Hooke is unable to give satisfying results.

In [ ]:
factor.Set(8)
gfu = SolveHooke(mesh)
Draw(Det(Id(2)+Grad(gfu)), mesh, deformation=gfu)

Next, we consider the St.Venant-Kirchhoff material law, which is geometrically nonlinear. It therefore delivers correct (meaning more realistic) results for larger deformation, but it is unable the prevent elements to be compressed to zero or even pressed through each other. If this happens, Newton does not converge anymore.

In [ ]:
aVK = BilinearForm(fes, symmetric=True)
aVK += Variation((StVenantKirchhoff(u)-force*u)*dx)

gfuVK = GridFunction(fes)

scene = Draw (Det(Id(2)+Grad(gfuVK)), mesh, deformation=gfuVK)

Newton's method with load-stepping algorithm using the NGSolve intern Newton solver.

In [ ]:
numsteps=10
with TaskManager():
    for i in range(numsteps):
        factor.Set(5*(i+1)/numsteps)
        print("loadstep = ", 5*(i+1)/numsteps)
        (result,it) = solvers.Newton(aVK, gfuVK, maxerr=1e-9, printing=False, maxit=50, inverse="sparsecholesky")
        if result == 0:
            scene.Redraw()
        if result != 0:
            input("Did not converge")
            scene.Redraw()
            break

The fully nonlinear material law of Hooke is able to handle large deformations including compression of elements. Its energy goes to infinity if the volume goes to zero. Therefore, the elements are prevented to degenerate or press through each other.

In [ ]:
gfuNH.vec[:] = 0
scene = Draw (Det(Id(2)+Grad(gfuNH)), mesh, deformation=gfuNH)
numsteps=10
with TaskManager():
    for i in range(numsteps):
        factor.Set(10*(i+1)/numsteps)
        print("loadstep = ", 10*(i+1)/numsteps)
        (result,it) = solvers.Newton(aNH, gfuNH, maxerr=1e-9, printing=False, maxit=30, inverse="sparsecholesky")
        if result == 0:
            scene.Redraw()
        else:
            break